# Environment Preparation

In [1]:
!pip install ../input/dlib-whl/dlib-19.19.0-cp36-cp36m-linux_x86_64.whl
!dpkg -i  /kaggle/input/ffmpeg-debs/ffmpeg/*.deb

Processing /kaggle/input/dlib-whl/dlib-19.19.0-cp36-cp36m-linux_x86_64.whl
Selecting previously unselected package ffmpeg.
(Reading database ... 59211 files and directories currently installed.)
Preparing to unpack .../ffmpeg_7%3a3.2.14-1~deb9u1_amd64.deb ...
Unpacking ffmpeg (7:3.2.14-1~deb9u1) ...
Selecting previously unselected package libass5:amd64.
Preparing to unpack .../libass5_1%3a0.13.4-2_amd64.deb ...
Unpacking libass5:amd64 (1:0.13.4-2) ...
Selecting previously unselected package libavc1394-0:amd64.
Preparing to unpack .../libavc1394-0_0.5.4-4+b1_amd64.deb ...
Unpacking libavc1394-0:amd64 (0.5.4-4+b1) ...
Selecting previously unselected package libavcodec57:amd64.
Preparing to unpack .../libavcodec57_7%3a3.2.14-1~deb9u1_amd64.deb ...
Unpacking libavcodec57:amd64 (7:3.2.14-1~deb9u1) ...
Selecting previously unselected package libavdevice57:amd64.
Preparing to unpack .../libavdevice57_7%3a3.2.14-1~deb9u1_amd64.deb ...
Unpacking libavdevice57:amd64 (7:3.2.14-1~deb9u1) ...
Selec

Selecting previously unselected package libssh-gcrypt-4:amd64.
Preparing to unpack .../libssh-gcrypt-4_0.7.3-2+deb9u2_amd64.deb ...
Unpacking libssh-gcrypt-4:amd64 (0.7.3-2+deb9u2) ...
Selecting previously unselected package libswresample2:amd64.
Preparing to unpack .../libswresample2_7%3a3.2.14-1~deb9u1_amd64.deb ...
Unpacking libswresample2:amd64 (7:3.2.14-1~deb9u1) ...
Selecting previously unselected package libswscale4:amd64.
Preparing to unpack .../libswscale4_7%3a3.2.14-1~deb9u1_amd64.deb ...
Unpacking libswscale4:amd64 (7:3.2.14-1~deb9u1) ...
Selecting previously unselected package libtbb2:amd64.
Preparing to unpack .../libtbb2_4.3~20150611-2_amd64.deb ...
Unpacking libtbb2:amd64 (4.3~20150611-2) ...
Selecting previously unselected package libtheora0:amd64.
Preparing to unpack .../libtheora0_1.1.1+dfsg.1-14+b1_amd64.deb ...
Unpacking libtheora0:amd64 (1.1.1+dfsg.1-14+b1) ...
Selecting previously unselected package libtwolame0:amd64.
Preparing to unpack .../libtwolame0_0.3.13-2_a

In [2]:
import cv2
import numpy as np
import os
import dlib
import glob
import pandas as pd
from matplotlib import pyplot as plt
import pickle
from tqdm import tqdm
from pathlib import Path
import traceback

pd.options.display.max_columns = 500
pd.options.display.max_rows = 2000

In [3]:
detector = dlib.get_frontal_face_detector()

# Variable Declaration

In [4]:
test_video_path = "/kaggle/input/deepfake-detection-challenge/test_videos/"
train_video_path = "/kaggle/input/deepfake-detection-challenge/train_sample_videos/"

# Utils Function

In [5]:
def split_video(path_to_video):
    path = Path(path_to_video)
    os.system(f"mkdir {path.stem}")
    os.system(f"ffmpeg -i {path_to_video} -r 1/1 {path.stem}/{path.stem}%03d.jpg")

In [6]:
def extract_face_from_img(path_to_img, detector, save_dir="."):
    faces_path = []
    try:
        img = cv2.imread(path_to_img)
        h, w, d = img.shape
        dets = detector(img, 1)
        path = Path(path_to_img)
        if len(dets) > 0:
            # 查找脸部位置
#             print("Number of faces detected: {}".format(len(dets)))
            for i, face in enumerate(dets):
                #保存脸部图片,注意这里+ - 是根据dlib对脸部的定位来的，我这里选的50很适合我这个视频
                # print(f"{max(face.top()-50, 0)}:{min(face.bottom()+50, h-1)},{max(face.left()-50, 0)}:{min(face.right()+50, w-1)}")
                crop_size = (224,224)
                face_region = img[max(face.top()-50, 0):min(face.bottom()+50, h),max(face.left()-50, 0):min(face.right()+50, w)]
                face_region = cv2.resize(face_region, crop_size, interpolation = cv2.INTER_CUBIC)
                save_path = f"{save_dir}/{path.stem}.jpg"
                cv2.imwrite(save_path, face_region)
                faces_path.append(save_path)
        else:
#             print("no face detected")
            pass
    except:
        traceback.print_exc()
    return faces_path

In [7]:
def extract_face_from_dir(img_dir, save_dir=".", count=3):
    faces_path = []
    for i, img in enumerate(os.listdir(img_dir), 1):
        img_path = f"{img_dir}/{img}"
#         print(i, img)
        imgs = extract_face_from_img(img_path, detector=detector, save_dir=save_dir)
        faces_path.extend(imgs)
        if len(faces_path) >= count:
            break
    return faces_path

In [8]:
# from https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/

def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape)

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (x.max()-x.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]
    radial_prof = tbin / nr

    return radial_prof

In [9]:
def get_psd1D(path_to_img, epsilon = 1e-8):
    img = cv2.imread(path_to_img,0)
    # Calculate FFT
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    fshift += epsilon
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = azimuthalAverage(magnitude_spectrum)
    return psd1D

# Predict

In [10]:
split_video("/kaggle/input/deepfake-detection-challenge/test_videos/ahjnxtiamx.mp4")

In [11]:
extract_face_from_dir("ahjnxtiamx")

['./ahjnxtiamx005.jpg', './ahjnxtiamx008.jpg', './ahjnxtiamx009.jpg']

In [12]:
#包导入
%matplotlib inline
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC

In [13]:
def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [14]:
model = load_object("../input/svm-model/svm4.pkl")

/opt/conda/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator SVC from version 0.21.0 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
df = pd.DataFrame(columns=['filename', 'label'])
count = 1
for i, video in tqdm(enumerate(os.listdir(test_video_path))):
    print(i, video)
    split_video(f"{test_video_path}/{video}")
    imgs = extract_face_from_dir(video.split(".")[0], count=count)
#     print(imgs)
    if len(imgs)<1:
        proba = 1
    else:
        probas = []
        for img in imgs:
            try:
                psd1D = get_psd1D(img)
                psd1D = psd1D[:156].reshape(-1, 156)
                probas.append(model.predict_proba(psd1D)[0][0])
            except:
                pass
        proba = np.mean(probas)
    print(proba)
    df.loc[i] = [video, proba]
df

0it [00:00, ?it/s]

0 iorbtaarte.mp4


1it [00:04,  4.57s/it]

0.4759928826432183
1 vnlzxqwthl.mp4


2it [00:09,  4.57s/it]

...

399it [49:44,  5.99s/it]

0.2891364019287707
399 mwwploizlj.mp4


400it [49:48,  7.47s/it]

0.529998468531923


,filename,label
0,iorbtaarte.mp4,0.475993
1,vnlzxqwthl.mp4,0.434028
2,gqnaxievjx.mp4,1.000000
3,sjkfxrlxxs.mp4,0.815537
4,eppyqpgewp.mp4,0.613892
5,oaguiggjyv.mp4,0.705496
6,ahjnxtiamx.mp4,0.593453
7,snlyjbnpgw.mp4,1.000000
8,hzssdinxec.mp4,0.846460
9,honxqdilvv.mp4,0.826029


In [16]:
!rm -rf /kaggle/working/*

In [17]:
df.to_csv("submission.csv", index=False)